In [1]:
import six
import numpy as np
import os  # needed navigate the system to get the input data
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import sys

########################### Radiomics package
import radiomics
from radiomics import featureextractor    
import SimpleITK as sitk
import logging


In [2]:
##########################
def catch_features(imagePath,maskPath):
    
    if imagePath is None or maskPath is None:
        raise Exception('imagePath is None or maskPath is None!')
    image_1 = sitk.ReadImage(imagePath)
    label_1 = sitk.ReadImage(maskPath)
    
    ###matching of CAD segmentation masks
    t2_origin = image_1.GetOrigin()
    mask_origin = label_1.GetOrigin()
    t2_spacing = image_1.GetSpacing()
    mask_spacing = label_1.GetSpacing()
    # Check if the origins are different
    if not np.isclose(np.array(t2_origin), np.array(mask_origin),atol=1e-1).all():
        # Set the mask origin to be the same as the T2 origin
        label_1.SetOrigin(t2_origin)

    if not np.isclose(np.array(t2_spacing), np.array(mask_spacing),atol=1e-1).all():
        # Set the mask origin to be the same as the T2 origin
        label_1.SetSpacing(t2_spacing)
    
    # Perform dilation on the mask using the structuring element
    # label_1 = sitk.BinaryDilate(label_1, [5,5,5])

    # Instantiate the extractor
    extractor = featureextractor.RadiomicsFeatureExtractor(geometryTolerance=1e-1)
    extractor.enableAllFeatures()
    extractor.enableAllImageTypes()
    
    result_1 = extractor.execute(image_1, label_1)
    
    # Make an array of the values
    feature_val = np.array([])
    feature_name = np.array([])

    for key, value in six.iteritems(result_1):
        if key.startswith("original_"):
            feature_val = np.append(feature_val, value)
            feature_name = np.append(feature_name, key)
            
    return feature_val, feature_name



In [6]:
image_dir = "/data/Lanhong/Pancreas_radiomics/MICCAI/t1/preprocessed"
mask_dir = "/data/Lanhong/Pancreas_radiomics/MICCAI/gt_segmentation_t1/cropped"
# image_dir = "/data/Lanhong/Pancreas_radiomics/MICCAI/t2/preprocessed"
# mask_dir = "/data/Lanhong/Pancreas_radiomics/MICCAI/gt_segmentation/cropped"

n = 107

save_file = np.empty([1,n])
id_list = np.array([])
patient_list = os.listdir(image_dir)


In [8]:
count =1
for patient in patient_list:
    patient = patient.split('.')[0]
    print(count,': Extracting:',patient)
    imagePath = os.path.join(image_dir,patient+".nii.gz") # t2 & t1 are both compatible with this format
    maskPath = os.path.join(mask_dir,patient+".nii.gz") 
    if not os.path.exists(maskPath):
        maskPath = os.path.join("/data/Lanhong/Pancreas_radiomics/MICCAI/gt_segmentation/cropped",patient+".nii.gz")
    # print(imagePath,maskPath)
    if os.path.exists(imagePath) and os.path.exists(maskPath):
        # try: 
        #     features, name = catch_features(imagePath, maskPath)   #feature extraction
        # except:
        #     print(f'img {patient} cannot be extracted. Moving to next.')
        #     continue
        features, name = catch_features(imagePath, maskPath)   #feature extraction
        features = np.array(features).reshape([1,n])    
        id_list = np.append(id_list,patient.split('.')[0])
        np.concatenate(([[patient]],features),axis=1)
        save_file = np.append(save_file,features,axis=0)
        print(count,': Extracted:',patient)
        count+=1

save_file = np.delete(save_file,0,0)
save_file_df = pd.DataFrame(save_file)
save_file_df.index = id_list
save_file_df.columns= name
save_file_df.to_csv('df_t1.csv')
print('Done Bro')

# sys.stdout = open('output.txt', 'w')
# sys.stdout.close()

1 : Extracting: MCA15


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Calculating Local Binary Pattern in 2D, but extracting features in 3D. Use with caution!
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Could not load required package "scipy" or "trimesh", cannot implement filter LBP 3D
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is

1 : Extracted: MCA15
2 : Extracting: CAD-226


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Calculating Local Binary Pattern in 2D, but extracting features in 3D. Use with caution!
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Could not load required package "scipy" or "trimesh", cannot implement filter LBP 3D
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is

2 : Extracted: CAD-226
3 : Extracting: NU_Patient_0008


...

267 : Extracted: MCA02
268 : Extracting: CAD-252


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Calculating Local Binary Pattern in 2D, but extracting features in 3D. Use with caution!
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Could not load required package "scipy" or "trimesh", cannot implement filter LBP 3D
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is

268 : Extracted: CAD-252
269 : Extracting: CAD-262


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Calculating Local Binary Pattern in 2D, but extracting features in 3D. Use with caution!
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Could not load required package "scipy" or "trimesh", cannot implement filter LBP 3D
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is

269 : Extracted: CAD-262
Done Bro
